In [1]:
import pandas as pd
import chant21 as ch
from music21 import converter
import os

/Users/pedroteche/miniconda3/envs/my_tf_env/lib/python3.10/site-packages/chant21/gabc/converter.py:438: Music21DeprecationWarning: registerSubconverter was deprecated on v9 and will disappear at or after v10. use unregisterSubconverter with capital C
  converter.registerSubconverter(ConverterGABC)


In [2]:
# set root dir to the folder above the script
DIRNAME = "/Users/pedroteche/Documents/GitHub/cantus_ai/"
root_dir = os.path.dirname(DIRNAME)
# set working directory to the root dir
os.chdir(root_dir)

In [3]:
def extractPitches(chant):
    pitches = []
    for element in chant.flatten().notes:
        if element.isNote:
            pitches.append(element.pitch)
    return [p.nameWithOctave for p in pitches]

def extractPitchesVolpiano(volpiano_string):
    try:
        ch = converter.parse(volpiano_string, format = 'volpiano')
        return extractPitches(ch)
    except:
        return []
        

In [4]:
tb_chant = pd.read_csv('data/chant.csv')
tb_genre = pd.read_csv('data/genre.csv')
tb_chant = tb_chant.loc[tb_chant['volpiano'].notna()].copy().reset_index(drop = True)
tb_chant['volpiano_length'] = tb_chant['volpiano'].apply(len)
p1 = tb_chant['volpiano_length'].quantile(0.05)
p2 = tb_chant['volpiano_length'].quantile(0.95)
tb_chant = tb_chant.loc[(tb_chant['volpiano_length'] >= p1) & (tb_chant['volpiano_length'] <= p2)].copy().reset_index(drop = True)

In /Users/pedroteche/miniconda3/envs/my_tf_env/lib/python3.10/site-packages/matplotlib/mpl-data/stylelib/dark_background.mplstyle: Columns (16) have mixed types. Specify dtype option on import or set low_memory=False.


In [8]:
# Antiphons
genre_list = ["genre_a"]
type_chant =  (tb_chant['genre_id'].isin(genre_list))               
tb_antiphon = tb_chant.loc[type_chant].copy().reset_index(drop = True)
tb_antiphon['notes'] = tb_antiphon['volpiano'].apply(extractPitchesVolpiano)
tb_antiphon['melody_length'] = tb_antiphon['notes'].apply(len)
q1 = tb_antiphon['melody_length'].quantile(0.05)
tb_antiphon = tb_antiphon.loc[(tb_antiphon['melody_length'] >= q1)].copy().reset_index(drop = True)
tb_antiphon[['id', 'melody_length', 'notes', 'mode']].explode('notes').reset_index(drop = True).to_csv('output/antiphon_melodies.csv', index = False)

In [9]:
# Chants of the Hours
type_chant =  (tb_chant['office_id'].notna()) & (tb_chant['office_id'] != 'office_x')              
tb_office = tb_chant.loc[type_chant].copy().reset_index(drop = True)
tb_office['notes'] = tb_office['volpiano'].apply(extractPitchesVolpiano)
tb_office['melody_length'] = tb_office['notes'].apply(len)
q1 = tb_office['melody_length'].quantile(0.05)
tb_office = tb_office.loc[(tb_office['melody_length'] >= q1)].copy().reset_index(drop = True)
tb_office[['id', 'melody_length', 'notes', 'mode']].explode('notes').reset_index(drop = True).to_csv('output/office_melodies.csv', index = False)

In [ ]:
# Chants of the Mass
tb_genre = tb_genre.dropna(subset = ['mass_or_office'])
tb_genre['mass_or_office'] = tb_genre['mass_or_office'].map(lambda x: eval(str(x)))
tb_mass = tb_genre[tb_genre['mass_or_office'].map(lambda x: 'Mass' in x)].copy().reset_index(drop = True)
tb_mass = tb_mass.merge(tb_chant, on = 'genre_id', how = 'inner')
tb_mass['notes'] = tb_mass['volpiano'].apply(extractPitchesVolpiano)
tb_mass['melody_length'] = tb_mass['notes'].apply(len)
q1 = tb_mass['melody_length'].quantile(0.05)
tb_mass = tb_mass.loc[(tb_office['melody_length'] >= q1)].copy().reset_index(drop = True)
tb_mass[['id', 'melody_length', 'notes', 'mode']].explode('notes').reset_index(drop = True).to_csv('output/mass_melodies.csv', index = False)